In [ ]:
# Installazione delle librerie necessarie con FAISS invece di Chroma
!pip install -q python-telegram-bot openai langchain langchain-openai langchain-community "faiss-cpu>=1.7.4" google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client pypdf tiktoken

# Import delle librerie
import os
import tempfile
import asyncio
import nest_asyncio
from google.colab import drive, auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
import io
import logging
from concurrent.futures import ThreadPoolExecutor
import time
import random
from datetime import datetime

# LangChain imports
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate
from langchain.schema.messages import HumanMessage, SystemMessage

# Telegram imports
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, ContextTypes, filters
from telegram.error import NetworkError, Forbidden, BadRequest, TimedOut
from telegram.constants import ParseMode

# Configurazione logging più dettagliata
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)
logger = logging.getLogger(__name__)

# Monta Google Drive
drive.mount('/content/drive')

# Risolvi il problema del loop asyncio in Colab
nest_asyncio.apply()

# Variabile globale per la catena di conversazione
conversation_chain = None

# Sistema prompt predefinito
DEFAULT_SYSTEM_PROMPT = """
Sei AuraCareBot💫, un'assistente personale per chi soffre di emicrania.
Assicurati che le informazioni fornite siano accurate e affidabili.
Includi barzellette e battute, usa un pizzico di umorismo per rendere le conversazioni più piacevoli.
Evita risposte lunghe e liste generiche di consigli, e prima di fare la domanda successiva vai a capo.
Inserisci qualche emoji nella risposta.
Se parli di esperienze personali, fallo ricordando che sei un bot, quindi non puoi averle vissute veramente.
Importante: Personalizza i consigli facendo ogni volta una domanda all'utente su temi che influenzano il mal di testa, poi, dopo 3 domande, dai una risposta finale e congeda l'utente, inserendo anche una curiosità sull'emicrania.
"""

# Funzione per ottenere i dati meteo (da adattare in base all'API in uso)
import requests

class Openweather:
    base_url = 'http://api.weatherapi.com/v1'
    forecast_url = '/forecast.json?'
    APIkey = 'API_key'

    def get_data(self, coordinates: str):
        url = f"{self.base_url}{self.forecast_url}&key={self.APIkey}&q={coordinates}&days=4"
        response = requests.get(url)
        response.raise_for_status()
        return response.json()

    def get_coordinates(self, zip_code: int):
        openweather_key = 'openweather_key'
        url = f"http://api.openweathermap.org/geo/1.0/zip?zip={zip_code},IT&appid={openweather_key}"
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        return f"{data['lat']},{data['lon']}"

def get_weather_advice(zip_code: int) -> str:
    ow = Openweather()
    coords = ow.get_coordinates(zip_code)
    weather_data = ow.get_data(coords)
    city = weather_data.get("location", {}).get("name", "Città sconosciuta")
    print(f"città: {city}")

    # Utilizza i dati attuali dal JSON (campo "current")
    current = weather_data.get('current', {})
    temp = current.get('temp_c')
    condition_text = current.get('condition', {}).get('text', "").lower()
    print(f"Temperatura: {temp}°C")
    print(f"Condizione: {condition_text}")

    location = weather_data.get("location", {})
    print(f"location json: {location}")
    local_time = location.get("localtime", "Unknown")
    print(f"Orario locale: {local_time}")

    local_time_dt = datetime.strptime(local_time, "%Y-%m-%d %H:%M")
    weekday = local_time_dt.weekday()
    print(f"Orario locale: {local_time}")
    print(f"Giorno della settimana: {weekday}")

    local_dt = local_time_dt.strftime("%A") + local_time

    advice = ""
    if temp is not None:
        if temp > 30:
            advice += "Le temperature elevate possono peggiorare il mal di testa; ricordati di restare idratato. "
        elif temp < 10:
            advice += "Il freddo può aumentare la tensione; cerca di stare al caldo. "

    if "rain" in condition_text or "pioggia" in condition_text:
        advice += "La pioggia e l'umidità potrebbero intensificare i sintomi: valuta di rimanere in ambienti asciutti. "
    elif "drizzle" in condition_text:
        advice += "Una leggera pioviggine potrebbe causare fastidio: cerca di riposare in un ambiente confortevole. "
    elif "cloud" in condition_text:
        advice += "Il cielo nuvoloso può influire sul tuo umore; prova ad adottare attività rilassanti. "
    elif "sunny" in condition_text:
        advice += "Una giornata soleggiata è piacevole, ma non dimenticare di proteggerti e mantenerti idratato. "

    if not advice:
        advice = "Le condizioni meteo sembrano normali oggi."

    print(f"Advice: {advice}")

    return advice, city, local_dt

# Funzione semplificata per autenticare e accedere a Google Drive usando colab.auth
def authenticate_gdrive_simple():
    auth.authenticate_user()
    return build('drive', 'v3', cache_discovery=False)

# (Il resto delle funzioni per il caricamento dei documenti rimangono invariato...)
def find_documents_in_drive(service):
    query = "name = 'AuraCareBot' and mimeType = 'application/vnd.google-apps.folder' and trashed = false"
    results = service.files().list(q=query, spaces='drive', fields='files(id, name)').execute()
    telegram_items = results.get('files', [])
    if telegram_items:
        parent_id = telegram_items[0]['id']
        logger.info(f"Trovata cartella 'AuraCareBot' con ID: {parent_id}")
        query = f"name = 'files' and mimeType = 'application/vnd.google-apps.folder' and '{parent_id}' in parents and trashed = false"
        results = service.files().list(q=query, spaces='drive', fields='files(id, name)').execute()
        files_items = results.get('files', [])
        if files_items:
            files_folder_id = files_items[0]['id']
            logger.info(f"Trovata cartella 'files' all'interno di 'AuraCareBot' con ID: {files_folder_id}")
            query = f"'{files_folder_id}' in parents and (mimeType = 'application/pdf' or mimeType = 'text/plain') and trashed = false"
            results = service.files().list(q=query, spaces='drive', fields='files(id, name, mimeType)').execute()
            file_items = results.get('files', [])
            if file_items:
                logger.info(f"Trovati {len(file_items)} file in 'AuraCareBot/files'")
                for file in file_items:
                    logger.info(f"File trovato: {file.get('name')} (Tipo: {file.get('mimeType')})")
                return file_items
    logger.warning("Nessun file trovato nella cartella")
    return []

def download_files_from_drive(service, files, temp_dir):
    downloaded_files = []
    print(f"\nSCARICAMENTO DI {len(files)} FILE:")
    for i, item in enumerate(files, 1):
        file_id = item['id']
        file_name = item['name']
        mime_type = item['mimeType']
        file_path = os.path.join(temp_dir, file_name)
        try:
            request = service.files().get_media(fileId=file_id)
            with open(file_path, 'wb') as f:
                downloader = MediaIoBaseDownload(f, request)
                done = False
                while not done:
                    status, done = downloader.next_chunk()
            downloaded_files.append({
                'path': file_path,
                'name': file_name,
                'mime_type': mime_type
            })
            print(f"{i}. ✓ {file_name} scaricato con successo")
        except Exception as e:
            print(f"{i}. ✗ Errore nel download di {file_name}: {str(e)}")
    return downloaded_files

def load_documents(files):
    documents = []
    print("\nCARICAMENTO DEI DOCUMENTI:")
    for i, file in enumerate(files, 1):
        file_path = file['path']
        mime_type = file['mime_type']
        file_name = file['name']
        try:
            if mime_type == 'application/pdf':
                loader = PyPDFLoader(file_path)
                pdf_docs = loader.load()
                documents.extend(pdf_docs)
                print(f"{i}. ✓ PDF: {file_name} - {len(pdf_docs)} pagine caricate")
            elif mime_type == 'text/plain':
                loader = TextLoader(file_path)
                text_docs = loader.load()
                documents.extend(text_docs)
                print(f"{i}. ✓ TXT: {file_name} - documento caricato")
        except Exception as e:
            print(f"{i}. ✗ Errore nel caricamento di {file_name}: {str(e)}")
    return documents

def patch_numpy():
    try:
        import numpy as np
        if not hasattr(np, 'float_'):
            print("Applicazione patch per NumPy 2.0...")
            np.float_ = np.float64
            print("Patch applicata con successo!")
    except Exception as e:
        print(f"Errore nell'applicazione della patch per NumPy: {str(e)}")

def create_vectorstore(documents):
    print(f"\nDIVISIONE DI {len(documents)} DOCUMENTI IN CHUNK...")
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    splits = text_splitter.split_documents(documents)
    print(f"Creati {len(splits)} chunk dai documenti")
    print("\nCREAZIONE DEL VECTORSTORE FAISS...")
    try:
        embeddings = OpenAIEmbeddings()
        vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)
        print("✅ Vectorstore FAISS creato con successo")
        return vectorstore
    except Exception as e:
        print(f"❌ Errore nella creazione del vectorstore: {str(e)}")
        raise

# Memorizza il prompt di sistema corrente
current_system_prompt = DEFAULT_SYSTEM_PROMPT

def create_conversation_chain(vectorstore, system_prompt=None, weather_context=""):
    global current_system_prompt
    print("\nCREAZIONE DELLA CATENA DI CONVERSAZIONE RAG CON PROMPT PERSONALIZZATO...")
    if system_prompt:
        current_system_prompt = system_prompt
    print(f"Utilizzo del prompt di sistema: {current_system_prompt}")
    llm = ChatOpenAI(model='gpt-4o', temperature=0.9)
    memory = ConversationBufferMemory(
        memory_key='chat_history',
        return_messages=True
    )
    system_template = current_system_prompt + """

Sei un assistente che risponde alle domande in base ai documenti forniti.

Context: {context}

History: {chat_history}
"""
    PROMPT = ChatPromptTemplate.from_messages([
        ("system", system_template),
        ("human", "{question}")
    ])
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(),
        memory=memory,
        verbose=False,
        combine_docs_chain_kwargs={"prompt": PROMPT},
        chain_type="stuff"
    )
    print("✅ Catena di conversazione con prompt personalizzato creata con successo")
    return conversation_chain

# Modifica di process_query per passare il weather_context al prompt
async def process_query(query, weather_context):
    global conversation_chain
    try:
        # Combina la domanda con il contesto meteo
        combined_query = f"{query}\n\nMeteo: {weather_context}"
        logger.info(f"Domanda combinata: {combined_query}")
        loop = asyncio.get_event_loop()
        response = await loop.run_in_executor(
            ThreadPoolExecutor(),
            lambda: conversation_chain({'question': combined_query})
        )
        logger.info("Query elaborata con successo")
        return response['answer']
    except Exception as e:
        logger.error(f"Errore nell'elaborazione della query: {str(e)}")
        return f"Mi dispiace, si è verificato un errore nell'elaborazione della tua richiesta: {str(e)}"


# Funzione sicura per inviare messaggi che gestisce gli errori di rete
async def safe_send_message(update, text, max_retries=5):
    retries = 0
    delay = 1
    while retries < max_retries:
        try:
            MAX_MESSAGE_LENGTH = 4000
            if len(text) <= MAX_MESSAGE_LENGTH:
                await update.message.reply_text(text, parse_mode='HTML')
                return True
            else:
                parts = [text[i:i+MAX_MESSAGE_LENGTH] for i in range(0, len(text), MAX_MESSAGE_LENGTH)]
                for i, part in enumerate(parts):
                    await update.message.reply_text(f"Parte {i+1}/{len(parts)}:\n\n{part}")
                return True
        except (NetworkError, Forbidden, BadRequest, TimedOut) as e:
            retries += 1
            if retries >= max_retries:
                logger.error(f"Impossibile inviare il messaggio dopo {max_retries} tentativi: {str(e)}")
                return False
            sleep_time = delay * (2 ** (retries - 1)) + random.uniform(0, 1)
            logger.info(f"Errore di rete, riprovo tra {sleep_time:.2f} secondi... ({retries}/{max_retries})")
            await asyncio.sleep(sleep_time)
        except Exception as e:
            logger.error(f"Errore non recuperabile nell'invio del messaggio: {str(e)}")
            return False

# Nuovo comando /setcap per memorizzare il CAP dell'utente
async def set_cap(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    parts = update.message.text.split()
    if len(parts) < 2:
        await safe_send_message(update, "Per favore, fornisci il CAP. Es: /setcap 10100")
        return

    cap = parts[1]
    context.user_data['cap'] = cap
    try:
        cap_int = int(cap)
        weather_context, city, local_dt = get_weather_advice(cap_int)
        # Salva il weather_context nella cache per l'utente
        context.user_data['weather_context'] = weather_context
    except Exception as e:
        await safe_send_message(update, f"Errore nell'impostazione del CAP: {str(e)}")
        return

    await safe_send_message(update, f"Il tuo CAP è stato impostato a {cap}.")

    # Se esiste una query pendente, processala
    pending_query = context.user_data.get('pending_query')
    pending_query = pending_query + f".  {local_dt}. Importante: Rispondi inserendo esplicitamente il nome della città ({city}) e il meteo"
    print(f"pending query: {pending_query}")
    if pending_query:
        try:
            # Usa il weather_context già salvato
            response = await process_query(pending_query, context.user_data['weather_context'])
            await safe_send_message(update, response)
            del context.user_data['pending_query']
        except Exception as e:
            await safe_send_message(update, f"Errore nel riprendere la conversazione: {str(e)}")



# Funzione per caricare i documenti
def load_documents_from_drive(system_prompt=None):
    global conversation_chain
    try:
        print("\n=== CARICAMENTO DEI DOCUMENTI DA GOOGLE DRIVE ===")
        service = authenticate_gdrive_simple()
        files_metadata = find_documents_in_drive(service)
        if not files_metadata:
            print("❌ Nessun file trovato nella cartella AuraCareBot/files.")
            return False
        temp_dir = tempfile.mkdtemp()
        print(f"✓ Directory temporanea creata: {temp_dir}")
        files = download_files_from_drive(service, files_metadata, temp_dir)
        if not files:
            print("❌ Nessun file è stato scaricato correttamente.")
            return False
        documents = load_documents(files)
        if not documents:
            print("❌ Nessun documento valido trovato nei file.")
            return False
        vectorstore = create_vectorstore(documents)
        conversation_chain = create_conversation_chain(vectorstore, system_prompt)
        print("✅ DOCUMENTI CARICATI CON SUCCESSO!")
        print(f"✅ {len(files)} file e {len(documents)} documenti pronti per l'uso.")
        return True
    except Exception as e:
        logger.error(f"Errore durante il caricamento dei documenti: {str(e)}")
        print(f"❌ ERRORE: {str(e)}")
        return False

# Comando /start
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    global conversation_chain
    if conversation_chain is None:
        await safe_send_message(
            update,
            "Ciao! Sono Aura💫, la tua assistente personale per l'emicrania.\nPer caricare i documenti, usa il comando /reload."
        )
    else:
        await safe_send_message(
            update,
            """Ciao! Sono Aura💫, la tua assistente personale per l'emicrania. Posso aiutarti a capire i tuoi sintomi, darti consigli pratici, o semplicemente tenerti compagnia quando il mal di testa si fa sentire.\nCome ti senti oggi?"""
        )

# Comando /scan per scansionare Google Drive
async def scan(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    await safe_send_message(update, "Sto scansionando Google Drive per trovare i tuoi documenti...")
    try:
        service = authenticate_gdrive_simple()
        files = find_documents_in_drive(service)
        if files:
            file_list = "\n".join([f"- {file['name']} ({file['mimeType']})" for file in files])
            await safe_send_message(
                update,
                f"Ho trovato {len(files)} file nella cartella:\n\n{file_list}\n\nUsa /reload per caricarli nel sistema."
            )
        else:
            await safe_send_message(
                update,
                "Non ho trovato file PDF o TXT nella cartella. Assicurati che i file siano stati caricati correttamente."
            )
    except Exception as e:
        logger.error(f"Errore durante la scansione di Google Drive: {str(e)}")
        await safe_send_message(
            update,
            f"Si è verificato un errore durante la scansione di Google Drive: {str(e)}"
        )

# Comando /reload per ricaricare i documenti
async def reload(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    global conversation_chain, current_system_prompt
    await safe_send_message(update, "Sto caricando i documenti dalla cartella AuraCareBot/files...")
    try:
        success = load_documents_from_drive(current_system_prompt)
        if success:
            await safe_send_message(
                update,
                "Documenti caricati con successo! Ora puoi farmi domande sui documenti."
            )
        else:
            await safe_send_message(
                update,
                "Si è verificato un errore nel caricamento dei documenti. Controlla la console di Colab per i dettagli."
            )
    except Exception as e:
        logger.error(f"Errore durante il caricamento dei documenti: {str(e)}")
        await safe_send_message(
            update,
            f"Si è verificato un errore durante il caricamento dei documenti: {str(e)}"
        )

# Comando /list per elencare i file trovati
async def list_files(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    await safe_send_message(update, "Sto cercando i file nella cartella AuraCareBot/files...")
    try:
        service = authenticate_gdrive_simple()
        files = find_documents_in_drive(service)
        if not files:
            await safe_send_message(
                update,
                "Nessun file PDF o TXT trovato nella cartella AuraCareBot/files."
            )
            return
        file_list = "\n".join([f"- {file['name']} ({file['mimeType']})" for file in files])
        await safe_send_message(
            update,
            f"Ho trovato i seguenti file nella cartella AuraCareBot/files:\n\n{file_list}"
        )
    except Exception as e:
        logger.error(f"Errore durante l'elenco dei file: {str(e)}")
        await safe_send_message(
            update,
            f"Si è verificato un errore durante l'elenco dei file: {str(e)}"
        )

# Comando /help per mostrare i comandi disponibili
async def help_command(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    help_text = """
📚 *COMANDI DISPONIBILI* 📚

/start - Avvia il bot e ricevi un messaggio di benvenuto
/scan - Scansiona Google Drive per trovare i file disponibili
/reload - Carica o ricarica i documenti nel sistema RAG
/list - Mostra l'elenco dei file trovati in Google Drive
/setstyle - Imposta lo stile del bot (es. /setstyle "Act as a poet and write in rhymes")
/setcap - Imposta il tuo CAP (es. /setcap 10100)

Per utilizzare il bot:
1. Assicurati di avere file PDF o TXT nella cartella 'AuraCareBot/files' su Google Drive.
2. Usa il comando /reload per caricare i documenti.
3. Imposta il tuo CAP con /setcap se non è già in memoria.
4. Fai domande sui documenti per ottenere risposte basate sul loro contenuto, integrando il contesto meteo.
    """
    await safe_send_message(update, help_text)

# Comando /setstyle per impostare lo stile del bot
async def set_style(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    global conversation_chain, current_system_prompt
    message_text = update.message.text
    system_prompt = message_text[9:].strip()
    if not system_prompt:
        await safe_send_message(
            update,
            "Per favore, fornisci un prompt di sistema. Esempio:\n\n"
            "/setstyle Act as a world class Poker Gamer. Use italian language. Ends your answers with a reference to the beauty Poker and gambling."
        )
        return
    current_system_prompt = system_prompt
    if conversation_chain is not None:
        try:
            retriever = conversation_chain.retriever
            conversation_chain = create_conversation_chain(retriever.vectorstore, system_prompt)
            await safe_send_message(
                update,
                f"Stile impostato con successo! Il nuovo prompt di sistema è:\n\n\"{system_prompt}\""
            )
        except Exception as e:
            logger.error(f"Errore nell'impostazione dello stile: {str(e)}")
            await safe_send_message(
                update,
                f"Si è verificato un errore nell'impostazione dello stile: {str(e)}"
            )
    else:
        await safe_send_message(
            update,
            f"Stile impostato con successo, ma i documenti non sono ancora caricati. Usa /reload per caricarli con il nuovo stile:\n\n\"{system_prompt}\""
        )

# Gestione dei messaggi: qui verifichiamo se l'utente ha già impostato il CAP; se non lo ha fatto, lo chiediamo.
async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    global conversation_chain

    if conversation_chain is None:
        await safe_send_message(
            update,
            "I documenti non sono ancora stati caricati. Usa il comando /reload per caricarli."
        )
        return

    cap = context.user_data.get('cap')
    if not cap:
        context.user_data['pending_query'] = update.message.text
        await safe_send_message(
            update,
            "L'emicrania può essere influenzata dal meteo attuale. Se vuoi, posso fornirti informazioni specifiche basate sul meteo della zona in cui ti trovi. Inserisci il tuo CAP scrivendo il comando /setcap seguito dal tuo codice postale."
        )
        return

    try:
        cap_int = int(cap)
    except Exception as e:
        await safe_send_message(
            update,
            "Il CAP inserito non è valido. Per favore, utilizza il comando /setcap seguito da un CAP numerico."
        )
        return

    # Usa il weather_context memorizzato se disponibile
    if 'weather_context' in context.user_data:
        weather_context = context.user_data['weather_context']
    else:
        # In alternativa, se per qualche motivo non è presente, lo recupera e lo salva
        weather_context = get_weather_advice(cap_int)
        context.user_data['weather_context'] = weather_context

    query = update.message.text
    try:
        response = await process_query(query, weather_context)
        await safe_send_message(update, response)
    except Exception as e:
        logger.error(f"Errore nella gestione del messaggio: {str(e)}")
        await safe_send_message(
            update,
            f"Si è verificato un errore nell'elaborazione della tua richiesta: {str(e)}"
        )


# Gestore di errori per l'applicazione
async def error_handler(update: object, context: ContextTypes.DEFAULT_TYPE) -> None:
    logger.error(f"L'aggiornamento {update} ha causato l'errore: {context.error}")

# Funzione principale
def main():
    telegram_token = os.environ.get('AuraCareBot_TOKEN')
    if not telegram_token:
        raise ValueError("AuraCareBot_TOKEN non trovato nelle variabili d'ambiente")
    if not os.environ.get('OPENAI_API_KEY'):
        raise ValueError("OPENAI_API_KEY non trovato nelle variabili d'ambiente")
    patch_numpy()
    print("\n=== TENTATIVO DI CARICAMENTO AUTOMATICO DEI DOCUMENTI ALL'AVVIO ===")
    load_documents_from_drive()
    application = Application.builder().token(telegram_token).build()
    application.add_handler(CommandHandler("start", start))
    application.add_handler(CommandHandler("reload", reload))
    application.add_handler(CommandHandler("scan", scan))
    application.add_handler(CommandHandler("list", list_files))
    application.add_handler(CommandHandler("help", help_command))
    application.add_handler(CommandHandler("setstyle", set_style))
    application.add_handler(CommandHandler("setcap", set_cap))  # Nuovo comando per impostare il CAP
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))
    application.add_error_handler(error_handler)
    logger.info("Avvio del bot...")
    print("\n=== BOT TELEGRAM AVVIATO ===")
    application.run_polling(
        allowed_updates=Update.ALL_TYPES,
        drop_pending_updates=True,
    )

if __name__ == "__main__":
    print("Imposta le variabili d'ambiente prima di eseguire il bot")
    print("Esegui il seguente codice in una cella separata:")
    print("import os")
    print("os.environ['AuraCareBot_TOKEN'] = 'il_tuo_token_telegram'")
    print("os.environ['OPENAI_API_KEY'] = 'la_tua_chiave_openai'")
    telegram_token = os.environ.get('AuraCareBot_TOKEN')
    openai_key = os.environ.get('OPENAI_API_KEY')
    if telegram_token and openai_key:
        print("Variabili d'ambiente trovate, avvio del bot...")
        main()
    else:
        print("Imposta le variabili d'ambiente e poi esegui main() in una nuova cella")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Imposta le variabili d'ambiente prima di eseguire il bot
Esegui il seguente codice in una cella separata:
import os
os.environ['AuraCareBot_TOKEN'] = 'il_tuo_token_telegram'
os.environ['OPENAI_API_KEY'] = 'la_tua_chiave_openai'
Variabili d'ambiente trovate, avvio del bot...

=== TENTATIVO DI CARICAMENTO AUTOMATICO DEI DOCUMENTI ALL'AVVIO ===

=== CARICAMENTO DEI DOCUMENTI DA GOOGLE DRIVE ===
✓ Directory temporanea creata: /tmp/tmpz3sm8jjw

SCARICAMENTO DI 4 FILE:
1. ✓ Battute.pdf scaricato con successo
2. ✓ GuidaAllEmicrania.pdf scaricato con successo
3. ✓ Facts-About-Migraine-AMF.pdf scaricato con successo
4. ✓ Aura_RandomInfo.pdf scaricato con successo

CARICAMENTO DEI DOCUMENTI:
1. ✓ PDF: Battute.pdf - 2 pagine caricate
2. ✓ PDF: GuidaAllEmicrania.pdf - 3 pagine caricate
3. ✓ PDF: Facts-About-Migraine-AMF.pdf - 1 pagine caricate
4. ✓ PDF: Aura_RandomInfo.p